In [2]:
!pip install datasets
!pip install zstandard
!pip install psutil


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import psutil
import timeit
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk, interleave_datasets
from itertools import islice

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Big data

In [4]:
# This takes a few minutes to run, so go grab a tea or coffee while you wait :)
data_files = "https://the-eye.eu/public/AI/training_data/code_clippy_data/code_clippy_dup_data/train/data_0_time1625776258_default.jsonl.zst"
pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
pubmed_dataset

Extracting data files: 100%|██████████| 1/1 [00:47<00:00, 47.32s/it]
Generating train split: 292251 examples [01:04, 4496.89 examples/s] 


Dataset({
    features: ['text', 'meta'],
    num_rows: 292251
})

In [5]:
pubmed_dataset[1]

{'text': '/*! Frow CSS | MIT License | frowcss.com\n\n// Core variables\n@import variables\n\n// Reset and dependencies\n@import src/normalize\n\n// Core CSS\n@import src/flexbox\n@import src/scaffolding\n@import src/grid\n\n// Utility classes\n@import src/responsive-utilities\n@import src/utilities',
 'meta': {'repo_name': 'Beg-in/frow',
  'stars': '144',
  'repo_language': 'CSS',
  'file_name': '_grid.sass',
  'mime_type': 'text/plain'}}

In [6]:
# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 720.26 MB


In [7]:
print(f"Number of files in dataset : {pubmed_dataset.dataset_size}")
size_gb = pubmed_dataset.dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

Number of files in dataset : 4150013170
Dataset size (cache file) : 3.87 GB


In [8]:
code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

Iterated over 292251 examples (about 3.9 GB) in 6.2s, i.e. 0.626 GB/s


In [9]:
pubmed_dataset_streamed = load_dataset(
    "json", data_files=data_files, split="train", streaming=True
)

In [10]:
next(iter(pubmed_dataset_streamed))

{'text': '//\n// Frow Variables\n// --------------------------------------------------\n\n//== Feature Toggles\n//\n//## Select the Frow features needed\n\n//** Basics (anchor links, h1-h6, etc.)\n$frow-enable-html-basics:                true !default\n\n//** Columns\n$frow-enable-columns:                    true !default\n\n//** Form elements\n$frow-enable-buttons:                    true !default\n$frow-enable-forms:                      true !default\n\n//** Responsive design\n$frow-enable-responsive-design-classes:  true !default\n\n//** Utilities\n$frow-enable-box-model-shorcuts:         true !default\n$frow-enable-utilities:                  true !default\n\n//** Normalize.css\n$frow-enable-normalize:                  true !default\n\n\n\n//== Colors\n//\n//## Gray and colors for use across Frow\n\n$gray-lighter: #EAEAEA !default\n$gray-light: #9E9E9E !default\n$gray-dark: #888888 !default\n$gray-darker: #4F4F51 !default\n\n$off-white: #BBB !default\n$off-black: #333 !default\n\n

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x["text"]))
next(iter(tokenized_dataset))

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 169kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 2.78MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 52.0MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.72MB/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors


{'text': '//\n// Frow Variables\n// --------------------------------------------------\n\n//== Feature Toggles\n//\n//## Select the Frow features needed\n\n//** Basics (anchor links, h1-h6, etc.)\n$frow-enable-html-basics:                true !default\n\n//** Columns\n$frow-enable-columns:                    true !default\n\n//** Form elements\n$frow-enable-buttons:                    true !default\n$frow-enable-forms:                      true !default\n\n//** Responsive design\n$frow-enable-responsive-design-classes:  true !default\n\n//** Utilities\n$frow-enable-box-model-shorcuts:         true !default\n$frow-enable-utilities:                  true !default\n\n//** Normalize.css\n$frow-enable-normalize:                  true !default\n\n\n\n//== Colors\n//\n//## Gray and colors for use across Frow\n\n$gray-lighter: #EAEAEA !default\n$gray-light: #9E9E9E !default\n$gray-dark: #888888 !default\n$gray-darker: #4F4F51 !default\n\n$off-white: #BBB !default\n$off-black: #333 !default\n\n

In [12]:
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
next(iter(shuffled_dataset))

{'text': '<?php\n/**\n * @link http://www.yiiframework.com/\n * @copyright Copyright (c) 2008 Yii Software LLC\n * @license http://www.yiiframework.com/license/\n */\n\nnamespace yii\\i18n;\n\nuse Yii;\nuse yii\\base\\Component;\nuse yii\\base\\InvalidConfigException;\n\n/**\n * I18N provides features related with internationalization (I18N) and localization (L10N).\n *\n * I18N is configured as an application component in [[\\yii\\base\\Application]] by default.\n * You can access that instance via `Yii::$app->i18n`.\n *\n * @property MessageFormatter $messageFormatter The message formatter to be used to format message via ICU\n * message format. Note that the type of this property differs in getter and setter. See\n * [[getMessageFormatter()]] and [[setMessageFormatter()]] for details.\n *\n * @author Qiang Xue <qiang.xue@gmail.com>\n * @since 2.0\n */\nclass I18N extends Component\n{\n    /**\n     * @var array list of [[MessageSource]] configurations or objects. The array keys are 

In [13]:
dataset_head = pubmed_dataset_streamed.take(5)
list(dataset_head)

[{'text': '//\n// Frow Variables\n// --------------------------------------------------\n\n//== Feature Toggles\n//\n//## Select the Frow features needed\n\n//** Basics (anchor links, h1-h6, etc.)\n$frow-enable-html-basics:                true !default\n\n//** Columns\n$frow-enable-columns:                    true !default\n\n//** Form elements\n$frow-enable-buttons:                    true !default\n$frow-enable-forms:                      true !default\n\n//** Responsive design\n$frow-enable-responsive-design-classes:  true !default\n\n//** Utilities\n$frow-enable-box-model-shorcuts:         true !default\n$frow-enable-utilities:                  true !default\n\n//** Normalize.css\n$frow-enable-normalize:                  true !default\n\n\n\n//== Colors\n//\n//## Gray and colors for use across Frow\n\n$gray-lighter: #EAEAEA !default\n$gray-light: #9E9E9E !default\n$gray-dark: #888888 !default\n$gray-darker: #4F4F51 !default\n\n$off-white: #BBB !default\n$off-black: #333 !default\n\

In [14]:
# Skip the first 1,000 examples and include the rest in the training set
train_dataset = shuffled_dataset.skip(1000)
# Take the first 1,000 examples for the validation set
validation_dataset = shuffled_dataset.take(1000)

In [15]:
law_dataset_streamed = load_dataset(
    "json",
    data_files="https://the-eye.eu/public/AI/training_data/code_clippy_data/code_clippy_dup_data/train/data_2_time1625798686_default.jsonl.zst",
    split="train",
    streaming=True,
)
next(iter(law_dataset_streamed))

{'text': '# Meshbird [twitter](https://twitter.com/meshbird) [website](https://meshbird.com/)\n\nDistributed private networking\n\n# Notice\n\nWe are moved to the new place:\n\n(https://github.com/meshbird/meshbird)',
 'meta': {'repo_name': 'meshbird/meshbird_gg',
  'stars': '292',
  'repo_language': 'None',
  'file_name': 'README.md',
  'mime_type': 'text/plain'}}

In [16]:
combined_dataset = interleave_datasets([pubmed_dataset_streamed, law_dataset_streamed])
list(islice(combined_dataset, 2))

[{'text': '//\n// Frow Variables\n// --------------------------------------------------\n\n//== Feature Toggles\n//\n//## Select the Frow features needed\n\n//** Basics (anchor links, h1-h6, etc.)\n$frow-enable-html-basics:                true !default\n\n//** Columns\n$frow-enable-columns:                    true !default\n\n//** Form elements\n$frow-enable-buttons:                    true !default\n$frow-enable-forms:                      true !default\n\n//** Responsive design\n$frow-enable-responsive-design-classes:  true !default\n\n//** Utilities\n$frow-enable-box-model-shorcuts:         true !default\n$frow-enable-utilities:                  true !default\n\n//** Normalize.css\n$frow-enable-normalize:                  true !default\n\n\n\n//== Colors\n//\n//## Gray and colors for use across Frow\n\n$gray-lighter: #EAEAEA !default\n$gray-light: #9E9E9E !default\n$gray-dark: #888888 !default\n$gray-darker: #4F4F51 !default\n\n$off-white: #BBB !default\n$off-black: #333 !default\n\

## The Pile

In [ ]:
base_url = "https://the-eye.eu/public/AI/pile/"
data_files = {
    "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(30)],
    "validation": base_url + "val.jsonl.zst",
    "test": base_url + "test.jsonl.zst",
}
pile_dataset = load_dataset("json", data_files=data_files, streaming=True)
next(iter(pile_dataset["train"]))